In [56]:
import random
from typing import List

from pyrankvote import Candidate, Ballot
from pyrankvote import (
    instant_runoff_voting
)

from tqdm.notebook import tqdm

In [3]:
candidates = [Candidate(str(i + 1)) for i in range(3)]
candidates

[<Candidate('1')>, <Candidate('2')>, <Candidate('3')>]

In [61]:
def get_candidate_by_name(candidates: List[Candidate], name: str):
    for candidate in candidates:
        if candidate.name == name:
            return candidate

In [33]:
segment_one = [
    Ballot(ranked_candidates=[candidates[1], candidates[2], candidates[0]])
    for i 
    in range(4)
]
segment_two = [
    Ballot(ranked_candidates=[candidates[0], candidates[1], candidates[2]])
    for i 
    in range(4)
]
pivotal_voter = Ballot(ranked_candidates=[candidates[1], candidates[0], candidates[2]])

In [34]:
ballots = segment_one + [pivotal_voter] + segment_two

In [35]:
method = instant_runoff_voting
kwargs = {}

In [36]:
election_result = method(candidates, ballots, **kwargs)

In [37]:
print(election_result)

FINAL RESULT
  Candidate    Votes  Status
-----------  -------  --------
          3        5  Elected
          2        4  Rejected
          1        0  Rejected



In [65]:
def get_social_result(election_result):
    result_lines = str(election_result).split("\n")[3:-1]
    ranked_candidates = []
    for line in result_lines:
        parsed_line = line.strip().split(" ")
        candidate_name = parsed_line[0]
        ranked_candidates.append(get_candidate_by_name(candidates, candidate_name))
    return ranked_candidates
get_social_result(election_result)

[<Candidate('3')>, <Candidate('2')>, <Candidate('1')>]

In [27]:
winners = election_result.get_winners()
winners

[<Candidate('1')>]

In [66]:
get_social_result(election_result)

[<Candidate('3')>, <Candidate('2')>, <Candidate('1')>]

In [ ]:
# Algorithm to determine a "pivotal voter":
# 1. Go through the ranking of each ballot, count the pairs.
#    Data model should look something like this: 
#    {Candidate('1'), Candidate('2')}: 5 - 5 ballots prefer candidate 1 to candidate 2.
# 2. Choose the highest preferences, eliminate the reverse preferences.
# 3. If this does not match the outcome, there is an inconsistency (intransivity)
# 4. ???

In [30]:
perturbed_ballots = ballots
is_pivotal_voter_found = False
for i, ballot in tqdm(enumerate(ballots)):
    for j in tqdm(range(len(ballot.ranked_candidates) - 1)):
        perturbed_candidates = list(ballot.ranked_candidates)
        perturbed_candidates[j], perturbed_candidates[j + 1] = perturbed_candidates[j + 1], perturbed_candidates[j]
        perturbed_ballot = Ballot(perturbed_candidates)
        perturbed_ballots[i] = perturbed_ballot
        perturbed_result = method(candidates, ballots, **kwargs)
        perturbed_winners = perturbed_result.get_winners()
        if winners != perturbed_winners:
            print(f"Voter {i + 1} is pivotal. "
                  f"Changing {ballot} to {perturbed_ballot}, in turn, "
                  f"changes the outcome from {winners} to {perturbed_winners}")
            is_pivotal_voter_found = True
            break
    if is_pivotal_voter_found and not is_find_all_pivotal_voters:
        break

Voter 1 is pivotal. Changing <Ballot(2, 1, 3)> to <Ballot(1, 2, 3)>, in turn, changes the outcome from [<Candidate('1')>] to [<Candidate('3')>]




In [21]:
ballot

<Ballot(3, 1, 2)>

In [22]:
dir(ballot)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_all_candidate_objects',
 '_is_candidate_object',
 '_is_duplicates',
 'ranked_candidates']

In [23]:
ballot.ranked_candidates

(<Candidate('3')>, <Candidate('1')>, <Candidate('2')>)